### python variables are memory references

In [1]:
var = 10
print(id(var))
print(hex(id(var)))
string = 'hello world!'
print(id(string))
print(hex(id(string)))

10919712
0xa69f20
140043181835248
0x7f5e581b43f0


### reference counting

In [2]:
var = 140616357063792
var_common = 1

In [3]:
from sys import getrefcount
print(getrefcount(var), " <- create an extra reference, affect reference count") 
print(getrefcount(var_common))

2  <- create an extra reference, affect reference count
3365


In [4]:
from ctypes import c_long
print(c_long.from_address(id(var)).value, " <- does not affect reference count")

1  <- does not affect reference count


### gargabe collection

In [96]:
import ctypes
import gc

def in_gc(object_id):
    return object_id in [id(obj) for obj in gc.get_objects()]

In [97]:
class O:
    def refer(self, other):
        self.other = other
        
    @property
    def address(self):
        return 'self: {0}, other: {1}'.format(hex(id(self)), hex(id(self.other)))

In [98]:
gc.disable()
# create circular reference
o1 = O()
o2 = O()
o1.other = o2
o2.other = o1
print(o1.address)
print(o2.address)

self: 0x7f5e580f36d8, other: 0x7f5e580f3908
self: 0x7f5e580f3908, other: 0x7f5e580f36d8


In [99]:
o1.other.other == o1

True

In [100]:
o2_object_id = id(o2)
# drop o2
o2 = None
print(c_long.from_address(o2_object_id).value)

o1_object_id = id(o1)
# drop o1
o1 = None
print(c_long.from_address(o1_object_id).value)

1
1


In [101]:
print(in_gc(o1_object_id))
print(in_gc(o2_object_id))

True
True


In [102]:
gc.collect()
print(in_gc(o1_object_id))
print(in_gc(o2_object_id))

False
False


### variable re-assignment

In [46]:
from ctypes import c_long

a = 10
a_id = id(a)

In [47]:
print(hex(a_id))
print(c_long.from_address(a_id).value)

0xa69f20
165


In [48]:
a = a + 10

In [49]:
# old address
print(hex(a_id))
print(c_long.from_address(a_id).value)

0xa69f20
164


In [50]:
# new address
print(hex(id(a)))

0xa6a060


### shared reference

In [1]:
# immutable
a = 10
b = 10
print('a shared reference with b:',a is b)

a shared reference with b: True


In [2]:
b = 20
print('a shared reference with b:',a is b)

a shared reference with b: False


In [3]:
# mutable
l1 = [1, 2, 3]
l2 = [1, 2, 3]
l3 = l1
print('l1 shared reference with l2:', l1 is l2)
print('l1 shared reference with l3:', l1 is l3)

l1 shared reference with l2: False
l1 shared reference with l3: True


In [4]:
l3.append(4)
print('l1 shared reference with l3:', l1 is l3)

l1 shared reference with l3: True
